In [4]:
# Using Python 3.10
%pip install yfinance
%pip install scikit-learn
%pip install keras
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/1f/a1/7d2042050159619a190db874913a2bc70645f8ac677d442f9aab4d29153e/tensorflow-2.17.0-cp310-cp310-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/a3/61/8001b38461d751c

### Simple Model

This model uses a stacked Long Short Term Model (LSTM) architecture with dense layers. The LSTM model structure is suited for time series data. 

The input layer contains the inputs lookback and number of features.

The first LSTM contains 50 LSTM units

{TBD}

In [ ]:
# imports for the model

import numpy as np
import pandas as pd 
import yfinance as yf # to import the historical prices
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# 1. Data preparation

def prepare_data(data, lookback=30):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:(i + lookback)])
        y.append(data[i + lookback, -1]) # target price is the lookback
    return np.array(X), np.array(y)

In [ ]:
# 2. Build Model

def build_model(input_shape):
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

In [ ]:
# 3. Main function

def predict_options_prices(ticker='MSFT', start_date="2010-01-01", end_date="2023-01-01", lookback=30):
    # Loading historical data from yfinance
    # df = yf.download(ticker, start=start_date, end=end_date)
    stock = yf.Ticker(ticker)
    options_dates = stock.options # Get available options expiration dates
    if not options_dates:
        raise ValueError("No options data available for this specific ticker")
    
    # Additional features for option pricing
    df['underlying_price'] = df['Close']
    df['strike_price'] = df['underlying_price'] * 1.05
    df['time_to_expiry'] = 30
    df['risk_free_rate'] = 0.01
    df['implied_volatility'] = df['Close'].pct_change().rolling(window=20).std() * np.sqrt(252)

    # Drop NaN values
    df.dropna(inplace=True)

    # Define features and target
    features = ['underlying_price', 'strike_price', 'time_to_expiry', 'risk_free_rate', 'implied_volatility']
    target = 'underlying_price'